# Astro 250 final project 

Margaret Doyle

mddoyle@lbl.gov 

May 2022

# Assemble training data

In [1]:
import h5py 
from IPython.display import clear_output
import numpy as np
from scipy.ndimage.interpolation import rotate
import matplotlib.pyplot as plt

#Have function for loading in runs and creating 2d image
def load_runs(runs,bt, d):
    x = []; y = []; adu = []; ndrop = []; gmd = []; evts = []
    for run in runs: 
        print(run)
        if bt == 'LX45':
            if d ==1:
                fh = h5py.File('/reg/data/ana16/mfx/mfxlx4519/scratch/ffb/xes/det1/run-%03d.h5' %run, 'r')
                X  = fh['x'][()]; Y = fh['y'][()]; I = fh['adu'][()]; N  = fh['ndroplets'][()]
                E  = fh['pulse_energy'][()];   fh.close()
            else: 
                fh = h5py.File('/reg/data/ana16/mfx/mfxlx4519/scratch/ffb/xes/det2/run-%03d.h5' %run, 'r')
                X  = fh['x'][()]; Y = fh['y'][()]; I = fh['adu'][()]; N  = fh['ndroplets'][()]
                E  = fh['pulse_energy'][()];  fh.close()    
        if bt == 'LY64':
            if d ==1:
                fh = h5py.File('/cds/data/drpsrcf/mfx/mfxly6420/scratch/xes/det1/run-%03d.h5' %run, 'r')
                X  = fh['x'][()]; Y = fh['y'][()]; I = fh['adu'][()]; N  = fh['ndroplets'][()]
                E  = fh['pulse_energy'][()];  fh.close()    
            else:
                fh = h5py.File('/cds/data/drpsrcf/mfx/mfxly6420/scratch/xes/det2/run-%03d.h5' %run, 'r')
                X  = fh['x'][()]; Y = fh['y'][()]; I = fh['adu'][()]; N  = fh['ndroplets'][()]
                E  = fh['pulse_energy'][()];  fh.close()    
        if bt =='LY38': 
            if d ==1:
                fh = h5py.File('/reg/data/ana15/mfx/mfxly3820/scratch/ffb/xes/det1/run-%03d.h5' %run, 'r')
                X  = fh['x'][()]; Y = fh['y'][()]; I = fh['adu'][()]; N  = fh['ndroplets'][()]
                E  = fh['pulse_energy'][()];  fh.close()   
        if bt == 'LX43':
            if d ==1:
                fh = h5py.File('/reg/data/ana16/mfx/mfxlx4319/scratch/ffb/xes/run-%03d.h5' %run, 'r')
                X  = fh['x'][()]; Y = fh['y'][()]; I = fh['adu'][()]; N  = fh['ndroplets'][()]
                E  = fh['pulse_energy'][()];  fh.close() 
                

            
        
        x.append(X); y.append(Y); adu.append(I),ndrop.append(N); gmd.append(E); 
    return np.concatenate(x),np.concatenate(y),np.concatenate(adu),np.concatenate(ndrop),np.concatenate(gmd)

def create_img(x,y,adu,ndrop,adu_int,scale_signal,bt, det):
    xm,ym = bin_dim(bt)
    if scale_signal == True:
        ind,  = np.where((adu >= adu_int[0]) & (adu <= adu_int[1]))
        img, xedges, yedges = np.histogram2d(x[ind], y[ind], bins=[np.arange(xm), np.arange(ym)],weights=adu[ind])
    elif scale_signal == False:
        ind,  = np.where((adu >= adu_int[0]) & (adu <= adu_int[1]))
        img_1ph, xedges, yedges = np.histogram2d(x[ind], y[ind], bins=[np.arange(xm), np.arange(ym)])
        ind, = np.where((adu >= 2.0*adu_int[0]) & (adu <= 2.0*adu_int[1]))
        img_2ph, xedges, yedges = np.histogram2d(x[ind], y[ind], bins=[np.arange(xm), np.arange(ym)])
        img = img_1ph + 2.0*img_2ph # scale 2-photon image by factor 2
    det_geom(img,bt)
    
    if det ==1:
        img = rotate(img[:, :], -88); 
      
        img = np.fliplr(img)
    else: 
        img = rotate(img[:, :], 90); 
        img =  np.fliplr(img)
        img = np.fliplr(img)

    return img

def bin_dim(bt):
    if bt == 'LX45':
        return 710,774
    else:
        return 705,769
    
def det_geom(img,bt):
    return img


# Collect Mn foil training data 

In [2]:

%matplotlib notebook

indx = 0
allshots = []
label =[]
#32
x, y, adu, ndrop, gmd = load_runs([264, 9,10], 'LY38', 1)

nimages = len(ndrop)
adu_int = [30,2800]

#Loop over images 
for i in np.arange(nimages-1):

    x2 = x[indx:indx + ndrop[i]]
    y2 = y[indx:indx + ndrop[i]]
    adu2 = adu[indx:indx + ndrop[i]]
    ndrop2 = len(adu2)
    print( "%0.2f percent done" %(i / nimages * 100))
    clear_output(wait=True)   
    img2 = create_img(x2,y2,adu2,ndrop2,[30,2800],True,'LX45',1)

    #If signal is decent, append and use for training data 

    if np.sum(img2[200:600, 90:140])> 8000:
        allshots.append(img2[200:600, 90:140]) 

        label.append('Mn_foil_Kb')
    indx += ndrop[i]

        



99.95 percent done


# Collect MnCl2 training data 

In [3]:

%matplotlib notebook

indx = 0

#32
x, y, adu, ndrop, gmd = load_runs([55,54], 'LX43', 1)

nimages = len(ndrop)
adu_int = [30,2800]

#Loop over images 
for i in np.arange(nimages-1):

    x2 = x[indx:indx + ndrop[i]]
    y2 = y[indx:indx + ndrop[i]]
    adu2 = adu[indx:indx + ndrop[i]]
    ndrop2 = len(adu2)
    print( "%0.2f percent done" %(i / nimages * 100))
    clear_output(wait=True)   
    img2 = create_img(x2,y2,adu2,ndrop2,[30,2800],True,'LX43',1)

    #If signal is decent, append and use for training data 

    if np.sum(rotate(img2[:, :], 90)[100:500, 560:560+50])> 13000:
    
        allshots.append(rotate(img2[:, :], 90)[100:500, 560:560+50]) 
#         plt.figure()
#         plt.imshow(rotate(img2[:, :], 90)[100:500, 560:560+50])
#         plt.show()
        label.append('MnCl2_Kb')
    indx += ndrop[i]

        



99.97 percent done


# Collect Fe III standard training data 

In [4]:
#Fe III standard 
%matplotlib notebook

indx = 0
#32
x, y, adu, ndrop, gmd = load_runs([32, 297, 298], 'LX45', 1)

nimages = len(ndrop)
adu_int = [30,2800]

#Loop over images 
for i in np.arange(nimages-1):

    x2 = x[indx:indx + ndrop[i]]
    y2 = y[indx:indx + ndrop[i]]
    adu2 = adu[indx:indx + ndrop[i]]
    ndrop2 = len(adu2)
    print( "%0.2f percent done" %(i / nimages * 100))
    clear_output(wait=True)   
    img2 = create_img(x2,y2,adu2,ndrop2,[30,2800],True,'LX45',1)

    #If signal is decent, append and use for training data 

    if np.sum(img2[0:400, 150:200])> 10000:

        allshots.append(img2[0:400, 150:200]) 
        label.append('Fe_III_nitrate_Ka')
    indx += ndrop[i]

        



99.98 percent done


# Fe foil 

In [5]:
x, y, adu, ndrop, gmd = load_runs([71, 126, 150, 192], 'LY64', 1)
indx = 0
nimages = len(ndrop)
adu_int = [30,2800]

#Loop over images 
for i in np.arange(nimages-1):

    x2 = x[indx:indx + ndrop[i]]
    y2 = y[indx:indx + ndrop[i]]
    adu2 = adu[indx:indx + ndrop[i]]
    ndrop2 = len(adu2)
    print( "%0.2f percent done" %(i / nimages * 100))
    clear_output(wait=True)   
    img2 = create_img(x2,y2,adu2,ndrop2,[30,2800],True,'LY64',1)

    #If signal is decent, append and use for training data 
    if np.sum(img2[0:400, 90:140])> 5000:
  
        allshots.append(img2[0:400, 90:140]) ,254
        label.append('Fe_foil_Ka')
    indx += ndrop[i]
    
x, y, adu, ndrop, gmd = load_runs([254], 'LY64', 1)
indx = 0
nimages = len(ndrop)
adu_int = [30,2800]

#Loop over images 
for i in np.arange(nimages-1):

    x2 = x[indx:indx + ndrop[i]]
    y2 = y[indx:indx + ndrop[i]]
    adu2 = adu[indx:indx + ndrop[i]]
    ndrop2 = len(adu2)
    print( "%0.2f percent done" %(i / nimages * 100))
    clear_output(wait=True)   
    img2 = create_img(x2,y2,adu2,ndrop2,[30,2800],True,'LY64',1)

    #If signal is decent, append and use for training data 
    if np.sum(img2[0:400, 100:150])> 8000:
        
        allshots.append(img2[0:400, 100:150])
        label.append('Fe_foil_Ka')
    indx += ndrop[i]

99.69 percent done


# Fe III nitrate 

In [6]:

%matplotlib notebook

indx = 0
x, y, adu, ndrop, gmd = load_runs([255,256], 'LY64', 1)

nimages = len(ndrop)
adu_int = [30,2800]

#Loop over images 
for i in np.arange(nimages-1):

    x2 = x[indx:indx + ndrop[i]]
    y2 = y[indx:indx + ndrop[i]]
    adu2 = adu[indx:indx + ndrop[i]]
    ndrop2 = len(adu2)
    print( "%0.2f percent done" %(i / nimages * 100))
    clear_output(wait=True)   
    img2 = create_img(x2,y2,adu2,ndrop2,[30,2800],True,'LX45',1)

    #If signal is decent, append and use for training data 

    if np.sum(img2[0:400, 150:200])> 10000:

        allshots.append(img2[0:400, 150:200]) 
                         
                                   
        label.append('Fe_III_nitrate_Ka')
    indx += ndrop[i]

        

99.97 percent done


# Ni II chloride

In [7]:
x, y, adu, ndrop, gmd = load_runs([32, 297, 298], 'LX45', 2)

indx = 0
nimages = len(ndrop)
adu_int = [30,2800]

#Loop over images 
for i in np.arange(nimages-1):

    x2 = x[indx:indx + ndrop[i]]
    y2 = y[indx:indx + ndrop[i]]
    adu2 = adu[indx:indx + ndrop[i]]
    ndrop2 = len(adu2)
    print( "%0.2f percent done" %(i / nimages * 100))
    clear_output(wait=True)   
    img2 = create_img(x2,y2,adu2,ndrop2,[30,2800],True,'LX45',1)

    #If signal is decent, append and use for training data 
    if np.sum(img2[797-400:797, 480:580-50]) > 10000:
        allshots.append(img2[797-400:797, 480:580-50]) 
        label.append('Ni_II_chloride_Ka')
    indx += ndrop[i]

    

99.98 percent done


# Ni II Nitrate 

In [8]:
x, y, adu, ndrop, gmd = load_runs([173,174], 'LX45', 2)

indx = 0
nimages = len(ndrop)
adu_int = [30,2800]

#Loop over images 
for i in np.arange(nimages-1):

    x2 = x[indx:indx + ndrop[i]]
    y2 = y[indx:indx + ndrop[i]]
    adu2 = adu[indx:indx + ndrop[i]]
    ndrop2 = len(adu2)
    print( "%0.2f percent done" %(i / nimages * 100))
    clear_output(wait=True)   
    img2 = create_img(x2,y2,adu2,ndrop2,[30,2800],True,'LX45',1)

    #If signal is decent, append and use for training data 
    if np.sum(img2[797-400:797,494-20:494+30])> 10000:
        allshots.append(img2[797-400:797,494-20:494+30]) 
        label.append('Ni_II_Nitrate_Ka')
    indx += ndrop[i]

99.94 percent done


In [9]:
x, y, adu, ndrop, gmd = load_runs([234], 'LY64', 2)

indx = 0
nimages = len(ndrop)
adu_int = [30,2800]

#Loop over images 
for i in np.arange(nimages-1):

    x2 = x[indx:indx + ndrop[i]]
    y2 = y[indx:indx + ndrop[i]]
    adu2 = adu[indx:indx + ndrop[i]]
    ndrop2 = len(adu2)
    print( "%0.2f percent done" %(i / nimages * 100))
    clear_output(wait=True)   
    img2 = create_img(x2,y2,adu2,ndrop2,[30,2800],True,'LX45',1)

    #If signal is decent, append and use for training data 
    if np.sum(img2[796-400:796, 660:660+50])> 16000:
        allshots.append(img2[796-400:796, 660:660+50]) 
        label.append('Ni_II_Nitrate_Ka')
    indx += ndrop[i]

99.94 percent done


# Ni II trisbipy

In [10]:
x, y, adu, ndrop, gmd = load_runs([176,177, 178], 'LX45', 2)
indx = 0
nimages = len(ndrop)
adu_int = [30,2800]

#Loop over images 
for i in np.arange(nimages-1):

    x2 = x[indx:indx + ndrop[i]]
    y2 = y[indx:indx + ndrop[i]]
    adu2 = adu[indx:indx + ndrop[i]]
    ndrop2 = len(adu2)
    print( "%0.2f percent done" %(i / nimages * 100))
    clear_output(wait=True)   
    img2 = create_img(x2,y2,adu2,ndrop2,[30,2800],True,'LX45',1)

    #If signal is decent, append and use for training data 
    if np.sum(img2[797-400:797, 480:580-50])> 10000:
        allshots.append(img2[797-400:797, 480:580-50]) 

        label.append('Ni_II_trisbipy_Ka')
    indx += ndrop[i]

99.98 percent done


# Ni foil 

In [11]:
x, y, adu, ndrop, gmd = load_runs([ 126], 'LY64', 2)
indx = 0
nimages = len(ndrop)
adu_int = [30,2800]

#Loop over images 
for i in np.arange(nimages-1):

    x2 = x[indx:indx + ndrop[i]]
    y2 = y[indx:indx + ndrop[i]]
    adu2 = adu[indx:indx + ndrop[i]]
    ndrop2 = len(adu2)
    print( "%0.2f percent done" %(i / nimages * 100))
    clear_output(wait=True)   
    img2 = create_img(x2,y2,adu2,ndrop2,[30,2800],True,'LY64',2)

    #If signal is decent, append and use for training data 
    if np.sum(img2[0:400, 555:605])> 10000:
        allshots.append(img2[0:400, 555:605]) 
        label.append('Ni_foil_Ka')
    indx += ndrop[i]

99.69 percent done


In [12]:
x, y, adu, ndrop, gmd = load_runs([9], 'LY64', 1)
indx = 0
nimages = len(ndrop)
adu_int = [30,2800]

#Loop over images 
for i in np.arange(nimages-1):

    x2 = x[indx:indx + ndrop[i]]
    y2 = y[indx:indx + ndrop[i]]
    adu2 = adu[indx:indx + ndrop[i]]
    ndrop2 = len(adu2)
    print( "%0.2f percent done" %(i / nimages * 100))
    clear_output(wait=True)   
    img2 = create_img(x2,y2,adu2,ndrop2,[30,2800],True,'LY64',1)

    #If signal is decent, append and use for training data 
    if np.sum(img2[0:400, 100:150])> -10000:
        
        allshots.append(img2[0:400, 100:150])
        label.append('Dark')
    indx += ndrop[i]

99.83 percent done


# Saving out and loading in data set 

In [13]:
hf = h5py.File('train_data.h5', 'w')
hf.create_dataset('image', data=allshots)
hf.create_dataset('label', data=label)
hf.close()

In [14]:
hf = h5py.File('train_data.h5', 'r')

In [15]:
images  = hf['image'][()]
labels = hf['label'][()]

In [16]:
print(f"labels shape: {labels.shape}")
print(f"images shape: {images.shape}")

labels shape: (19611,)
images shape: (19611, 400, 50)
